# Profitable Apps 
For this project we will assume the role of a data analyst for a mobile app development company.
**Our goal is to build a free app that generates revenue by new ad views.**  This implies that the more users who
see the add/install the app, the more revenue we will generate.


This project will involve analyzing available app store data to find predictors of an app being likely to attract
 new users.  Assuming we find good indicators for number of installs, we will attempt to build a predictive model that 
 given early ratings could tell us whether the app is likely to have a lot of installs or not.

## Opening and Exploring the Data
*We will be working with a dataset from 2018 as it was readily available.  It consists or approximately 10k+ apps from 
the Google App markets.  This dataset is available via Kaggle at the below link*
- [Google Play Store Data](https://www.kaggle.com/lava18/google-play-store-apps)


Using this sample of the true dataset should suffice at achieving our goal without making the investment to collect 
a larger dataset.  It is important to note the assumption that the marketplace hasn't changed in the time period in 
following the dataset.  In a production environment this would only serve as a proof of concept and an opportunity
to gain some domain knowledge on feature importance.

**Lets start by opening the data set**

In [11]:
# IMPORTS #
import pandas as pd
import numpy as np
import seaborn as sns
import os
import re
import jenkspy
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import binarize




google_raw = pd.read_csv(r'RawData/googleplaystore.csv')  # Read the csv file into a pandas Dataframe


One thing to always be conscious of is where your data is coming from.  In our case, the Google Dataset from Kaggle has 
a discussion section and one top discussion involves an inaccuracy in our data.  Apparently the 'Life Made Wi-Fi 
Touchscreen Photo Frame' app has invalid data.  In this case, since it is such a small sample of our data, we will just
remove it.

In [12]:
print(google_raw.loc[[10472]])  # After verifying we have the correct index, drop it using .drop
df = google_raw.drop(google_raw.index[10472])

KeyError: "None of [Int64Index([10472], dtype='int64')] are in the [index]"

## Cleaning the Data and Creating Features

Now that we have dropped the advised row of incorrect data, we should spend some time ourselves checking against common
issues associated with unknown datasets.  The first area to check is for duplicate entries in our dataset. Since our
dataset is defined at the app level, we can check this by checking for duplicate application names.

In [ ]:
dups = df[df.duplicated(['App'])].App  # Create a DF and then filter to a Series of all Duplicated App Names
sorted(set(dups))[:10]  # Convert to a set to remove duplicates, sort them and print out the first 10 for review

As we can see, there are quite a few duplicate entries in our dataset.  Now that we see a preview of some examples 
we can take a look at their raw data to see if and where their differences are.

In [ ]:
df[df.App == '10 Best Foods for You']  # Lookup the App by Name

In [ ]:
df[df.App == 'Instagram']  # Lookup the App by Name

Through some quick investigation it is clear that the area in which there are often differences in the duplicated apps 
is in the total number of reviews given to that app.  There are a few options here, but I believe the most efficient use
 of our time is to go with the App with the highest review count.  
 

In [ ]:
df.sort_values(by=['Reviews'], inplace=True, ascending=False)  # Sort rows by highest reviews to lowest
# Remove any duplicate App names, keeping the first occurrence
df.drop_duplicates(subset='App', keep='first', inplace=True)

# Check for dups again to confirm they were removed
dups = df[df.duplicated(['App'])].App
print(sorted(set(dups))[:10])

df[df.App == 'Instagram']  # Verify we removed all dups keeping the highest review count

As we can now see we have removed all duplicate App names from our Google dataset.  We've also displayed the results 
for instagram so that we can check we removed duplicates correctly.

The next step seems to be to start analyzing the data from our slightly cleaned set.  **However, if we keep the goal 
of this project in mind, we will remember that we do not care about paid or Non-English apps.**  Our goal is to make a 
free, english friendly app.  it is important to remove these other apps from our dataset early on so that we do not
consider their values as valid while cleaning and eventually feature engineering.

In [ ]:
def is_english(row: pd.DataFrame, met: str, flag: int) -> bool:
    """
    :param row: a row from a pandas dataFrame
    :param met: a column header to determine the column checking
    :param flag: an int provided to serve as the threshold for how many non-ASCII characters are allowed
    :return: a boolean that returns True if it is english
    
    This function uses python's built-in ord function to find out the corresponding number of each character.
    English characters (including punc) are encoded as ASCII standard, meaning each character should fall within 0 and
    127.  However there are instances where english apps use non ASCII characters (emojis, dashes, etc..)  Due to this 
    we will set a threshold for the number of non-ASCII characters the string can have.
    
    TODO: may be worth having the threshold provided systematically based on length of string, etc..
    """
    non_ascii = 0  # Establish counter
    for char in row[met]:  # For each character in the string
        if ord(char) > 127:  # If non-english
            non_ascii += 1  # add 1 to the counter
    
    if non_ascii > flag:  # Post Loop, if counter above threshold, return False
        return False
    else:
        return True
    
df = df[df.apply(is_english, axis=1, met='App', flag=3) == True]  # Apply is_enlgish function row by row on the App col
df.head()

Removing price is much simpler, as we have a categorical variable for Free Vs Paid in our dataset.  It is important in 
these situation to check your classifying variables.  So we should look to see if there are cases in which the App is 
labeled as paid, but appears to have no cost.

In [ ]:
# Check to ensure there are not invalid cases of this classification variable
invalid = df[(df.Price == '0') & (df.Type == 'Paid')]
print(invalid)

# Double Check that all we have remaining is "Free" apps
df = df[df.Type != 'Paid']
df.Type.unique()

To make further data exploration easier we will start by exploring the columns and the data types contained
in each column.

In [ ]:
def get_cat(series: pd.Series, empty_flag: bool) -> str:
    """
    :param series: a pandas series of data (should be of one data type)
    :param empty_flag: a boolean to call out an empty column (containing all NaNs)
    :return: a string that is the 'category of the datatype'
    
    This function will take a given series and return a generalized category for the datatype.
    This is necessary for readability as df.dtypes will return ungrouped types (i.e. float64, int64)
    as well as return some options that do not provide much insight (i.e. object)
    """
    if empty_flag:  # Catch Empty Columns
        return 'Empty'
    elif pd.api.types.is_numeric_dtype(series):
        return 'Numerical'
    elif pd.api.types.is_datetime64_dtype(series):
        return 'Date'
    elif pd.api.types.is_bool_dtype(series):
        return 'Boolean'
    elif pd.api.types.is_categorical_dtype(series):
        return 'Categorical'
    elif pd.api.types.is_string_dtype(series):
        return 'Text'
    else:  # Catches all others for review
        return 'Unknown'


def col_categories(data: pd.DataFrame):
    """
    :param data: a pandas Dataframe consisting of columns of data you want to explore their categories
    :return: None
    Prints the column name and data type of each column in the dataframe 
    """
    if data.dropna().empty:  # Catches Error
        raise ValueError('DataFrame Provided Contains is either empty or only contains NaNs') 
        # sys.exit(1)
    for col in data.columns:  # For Each Column
        if data[col].dropna().empty:  # Catch Empty Columns
            empty = True  # Set Flag
        else:
            empty = False  # Set Flag
        print(col, ': ', get_cat(data[col], empty))  # Print and Call Column Categories

In [ ]:
# Call col_categories to get an understanding of our Datasets 
print('Google Data')
col_categories(df)


Now that we have cleaned up our dataset a little, we can seem some areas where we may be suspicious of the data type 
returned to us.  For example, "Category" in the Google data could potentially be converted into a categorical variables. 
While making these conversions may not be the most crucial step we will do here today, it serves as a good step to 
ensure later on you can grab data by type. Even for metrics we may not end up using, it is reasonable to do some basic 
level of cleaning and converting so that this data can be exported and saved for future uses (which may need the other 
features)

**The first step is to confirm our assumptions on data types by looking at samples of the data.**

In [ ]:
# Look at the first few rows of our data
df.head()

*The user should be looking at more than the first 5 rows of data to get a good understanding of the data, this 
amount is just used for readability*

### As we expected, we have a few areas worth converting/cleaning:
- **Category**
    - From Text to Category
- **Size** - Seems like there are 2 Options 
    - Try converting to a numerical and impute the "Varies By Size"
    - Explore the distribution of this metric to see if we can create ranges and thus categories
- **Installs**
    - From Text to Categorical
- **Type**
    - From Text to Categorical
- **Content Rating**
    - From Text to Categorical
- **Genre**
    - Multiple Entry Text, split on ; - Will probably be worth to dummify the variable to turn them into multiple binary
    variables
- **Last Updated**
    - From Text to Date

*Current Version and Android Version Required are not something I will be working with as I think those categories are 
more complicated in their feature engineering, and we want to prioritize our time in the most effective way.*

In [ ]:
# Simple Text to Categorical Conversion
df['Category'] = pd.Categorical(df.Category)
df['Installs'] = pd.Categorical(df.Installs)
df['Type'] = pd.Categorical(df.Type)
df['Content Rating'] = pd.Categorical(df['Content Rating'])
df['Reviews'] = df['Reviews'].astype('int')

#  Recall our Original Col DataType Function
print('Google Data')
col_categories(df)

Now we have completed the one step conversion of data types, we can move onto the more complicated.  We will
start by analyzing the Size Category.  I believe the best process will be to first analyze the data we have for google.

By examining the data in the CSV file we can see they have a standard data structure format. We can use regular 
expressions to analyze this more.

In [ ]:
def patt_out(row: pd.DataFrame, met:str, patt:str, result:list):
    """
    :param row: row supplied by the apply function
    :param met: the column of interest from the row
    :param patt: Regex pattern that is the standard format
    :param result: a list to append to all of the non-standard format responses for investigation
    :return: None
    
    This function serves to find those values from a given row/column that do not fit a regex format supplied.
    This can be used to ensure we are not missing some odd values during out data cleaning.
    
    TODO: Convert to a bigData Approach using Pyspark for scalability 
    """
    pattern = re.compile(patt)
    if not pattern.match(row[met]):
        result.append(row.Size)
    return None


mismatch = list()  # Initiate a list to contain results
regexp = '(\d+(?:\.\d+)?([Mk])$)'  # Looks for any int or float
# Applies the pattern outlier function row by row
df.apply(patt_out, axis=1, met='Size', patt=regexp, result = mismatch)
mismatch = set(mismatch)  # Removes Duplicates by Converting to a Set
mismatch

**As you can see through our investigation of our data format for the Size column, everything follows the pattern:**
- A Numeric Followed by 'M' for megabyte or 'k' for kilobyte
- A Categorical Variable of 'Varies with device'

Now to really get this feature into a category datatype we have to convert those with k or M to their corresponding 
number of bytes (shared unit).

In [ ]:
def convert_bytes(val:str) -> float:
    """
    :param val: Values from a Series containing either a 'M' or 'k' to signify the unit of bytes
    :return: converted to bytes value as a float
    
    This function serves to determine the byte units implied by the string and convert to raw kilobytes for comparison
    """
    if val == 'Varies with device':
        return 0.0
    if val[-1] == 'M':
        return float(val[:-1]) * 1000  # Multiply by 1000 to convert to kb
    if val[-1] == 'k':
        return float(val[:-1])

sizes = df[df.Size != 'Varies with device'].Size  # Creates a series of all sizes we have data on
sizes = sizes.apply(convert_bytes)  # Converts the sizes to the same unit, kilobytes

sizes.describe()  # View General Descriptive Statistics

Now we can see some basic descriptive statistics from this metric, we can start to gain insight on how we
should categorize this variable.  Of interest we can see the average size is approx. 20,000 kb for our apps, however 
the variance in this metric is very high as well with a standard deviation of almost 22,000 kb.  Due to this I find it 
may be valuable to get a better look at the distribution of this metric.

In [ ]:
ax = sns.kdeplot(sizes, shade=False, legend=None)  # Create a Subplot to get the Kernel Density Estimation Plot
lines = ax.get_lines()[0]  # Grab the lin objects from the kde
x, y = lines.get_data()  # convert line into x & y coords

# Color fill the visual based on percentile - Default Interpolation
ax.fill_between(x, y, where = x <= np.percentile(sizes, 25), color='r', alpha=0.5)
ax.fill_between(x, y, where = (x > np.percentile(sizes, 25)) & 
                              (x < np.percentile(sizes, 75)), color='y', alpha=0.5)
ax.fill_between(x, y, where = x >= np.percentile(sizes, 75), color='g', alpha=0.5)

# Add vertical lines to signify the percentiles we shaded by
ax.vlines(x = np.percentile(sizes, 25), ymin=0, ymax=np.interp(np.percentile(sizes, 25), x, y))
ax.vlines(x = np.percentile(sizes, 75), ymin=0, ymax=np.interp(np.percentile(sizes, 75), x, y))
ax.vlines(x = np.percentile(sizes, 50), ymin=0, ymax=np.interp(np.percentile(sizes, 50), x, y), color='r')

#Add Data Labels for our lines to avoid confusion
ax.text(np.percentile(sizes, 25), np.interp(np.percentile(sizes, 25), x, y), '25th Perc.')
ax.text(np.percentile(sizes, 50), np.interp(np.percentile(sizes, 50), x, y), 'Median')
ax.text(np.percentile(sizes, 75), np.interp(np.percentile(sizes, 75), x, y), '75th Perc.')

ax.set_title('Google App Size Distribution')  # Add a title

As we can see in the above Kernel Density Plot (used as an approximation of the PDF) the size data is heavily skewed to 
left and smaller apps.  This will mean that there is much more variation in the upper percentiles when it comes to App 
size.  There are two main ways people tend to lean when presented with this type of situation.  
- That you have so many apps clustered together on the small side that we need additional categories on that side to 
see the true nuance.
- Since there is so much variance (large spread) at the top percentiles we will want to have extra categories on that 
side of the distribution to account for different behavior.

In cases like these I always find it beneficial to apply Fisher-Jenks algorithm first to help confirm the insight 
we've gained from our early analysis.  While this algorithm will find the natural breaking points for us in the data, 
we want to evaluate the distribution both with and without outliers.

In [ ]:
#Inner Quartile Range
IQR = (np.percentile(sizes, 75) - np.percentile(sizes, 25))  # Establish the Inner Quartile Range
upper_bound = np.percentile(sizes, 50) + (IQR * 1.5)
lower_bound = np.percentile(sizes, 50) - (IQR * 1.5)

sizes_trim = sizes[(sizes >= lower_bound) & (sizes <= upper_bound)]  # Trim off sizes outside our range

# Print Results of each Percentile
print(os.linesep, 'Outliers Not Removed')
for i in range(0, 9):
    print ((i + 1)*10, 'Percentile', np.percentile(sizes, (i +1)*10))

# Establish the natural breaking points using Fisher-Jenks
breaks = jenkspy.jenks_breaks(sizes, nb_class=5)
print(os.linesep, 'FJ breaks', breaks)  # Print the Break Points

# Print percentiles with outliers removed
print(os.linesep, 'Outliers Removed')
for i in range(0, 9):
    print ((i + 1)*10, 'Percentile', np.percentile(sizes_trim, (i +1)*10))

# Establish and print breaks using fisher-jenks algo with outliers removed 
breaks = jenkspy.jenks_breaks(sizes_trim, nb_class=5)
print(os.linesep, 'FJ breaks', breaks)


Using the inner quartile range (*1.5) method to remove outliers has removed approx. 12% of our apps based on size. 
With a quick scan we can see that essentially they were all on the upper limits, however this seems like a large 
percentage of outliers.

While it is unconventional, I think the appropriate action here is to keep outliers in but make a footnote of their 
inclusion.  In this case I will use the Fisher-Jenks results without outliers to build my categories, but then 
include the ranges when outliers are included, stating these are outliers.

In [ ]:
df['Size_Conv'] = df.Size.apply(convert_bytes)  # Convert bytes into a numeric column for investigation

# Establish ranges 
criteria = df.Size_Conv.between(0, 8.5), df.Size_Conv.between(8.5, 6401), df.Size_Conv.between(6401, 13001), \
           df.Size_Conv.between(13001, 22001), df.Size_Conv.between(22001, 33001), df.Size_Conv.between(33001, 46001), \
           df.Size_Conv.between(46001, 69001), df.Size_Conv.between(69001, 100001)
# Establish values associated with ranges
values = [1, 2, 3, 4, 5, 6, 7, 8]

#Apply the classification into it's own column
df['Size_Class'] = np.select(criteria, values, 0)
df['Size_Class'] = pd.Categorical(df.Size_Class)

Now we are down to the last two columns in our list for cleaning/converting.  First we will tackle 'Genre' which 
appears to be stored as a semi-colon separated list.  Our previous datatype search lets us know that it is stored 
as a string/text.  So, we will have to separate these genres and convert them into binary variables.  This is often 
referred to as 'One hot encoding".  Thankfully Pandas has a somewhat simple process for doing so.  We will also take 
the necessary steps to encode our other categorical predictors

In [ ]:
# Separate the genres column on ';' and then use the get_dummies to turn into binary variables
# Then we will rename each column as "Genre_" and whatever the genre names.  Last we concat this back to our
# original dataframe so we have all the data together
df = pd.concat([df, df.Genres.str.get_dummies(sep=';').rename(lambda c: 'Genre_' + c, axis='columns'),
                df.Category.str.get_dummies(sep=',').rename(lambda c: 'Category_' + c, axis='columns'),
                df['Content Rating'].str.get_dummies(sep=',').rename(lambda c: 'ContentRating_' + c, axis='columns')],
               axis=1)


# Check to ensure the genres were one hot encoded
df[df.App == 'Basketball FRVR - Shoot the Hoop and Slam Dunk!']

Now these have been encoded as binary variables, we gain the opportunity to feature engineer if a game has one or
multiple genres.

In [ ]:
df['multi_gen'] = df.loc[:, df.columns.str.startswith('Genre_')].sum(axis=1)  # Sum the Genres Column
df.loc[df.multi_gen < 2, 'multi_gen'] = 0  # Map single genres to False
df.loc[df.multi_gen >= 2, 'multi_gen'] = 1  # Map multi genres to true


df.head()

We will also create a column for rounding ratings to it's closest 0.5 score as this may end up being useful for
analysis later.

In [ ]:
df['rating_round'] = round(df.Rating * 2) / 2  # Rounds the rating to the closest 0.5x



Converting Last Updated to a date is fairly simple as long as you know the date format your data is currently in.
In this case we want to convert it to a datetime variable using to_datetime and the format = %d-%b-%y

In [ ]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'], format='%d-%b-%y')  # Convert text to Datetime
df['Last Updated'].head()  # Verify that the data has been converted correctly

***At this point our Google Play Market data has been cleaned/converted and stands ready for analysis***

We will export this to our RawData folder in case we need to reference it outside of this notebook.

In [ ]:
df.to_csv(r'RawData/google_cleaned.csv', index=False)  # Export df without index to CSV

## Building Labels to Define Success
If we are are going to complete any sort of feature selection or basic supervised learning 
action, we need to have labels to train to. 

In [ ]:
sns.countplot(x='Installs', data=df)  # Basic Count Plot of our Install Metric

As we can see, the installs variable may need to be cleaned up.  Since this is a categorical variable 
based of numerics, they have a natural order. 

Since the data does not seem to be standardized, we may need to create a dictionary to save the correct order from 
checking the original data.  We know there are 21 categories, so it is not too hard to order.  


In [ ]:
# Ordered list of installs
inst = ['0','0+','1+','5+','10+','50+','100+','500+','1,000+','5,000+','10,000+','50,000+','100,000+','500,000+',
        '1,000,000+','5,000,000+','10,000,000+','50,000,000+','100,000,000+','500,000,000+','1,000,000,000+']

installs = {}  # Init a Dict to hold order and count
for i, cat in enumerate(inst): # Loop through list
    installs[cat] = [i, 0]  # establish the keys and values (order) and an empty count
    
installs

Now we have a dictionary of install categories and it is ordered by size.  We will now add a second value to each key 
with their counts from the dataset.  This should serve as a better visual for us to make our labels from.

In [ ]:
for i in df.Installs:
    installs[i][1] += 1  # for each key we come across increment the count value by one
    
installs  # Preview installs

Now we can see the distribution of our install groups, too keep this simple we should maybe define our labels to 
be based off of a certain percentile of responses.  We know we have approx. 8,800 responses, so we can easily break 
this distribution into Quartiles (Approx. 2,200 responses per quartile).  We are okay with rough estimates here as 
we are just investigating the current market.  We need some way to define a 'true' label, so we start with basic and 
see our results. 

1,000+ & Below: 2249 Responses (25.4 perc)
5,000+ - 100,000+: 2751 Responses (56.4 perc)
500,000+ - 5,000,000+: 2495 Responses (84.6 perc)
10,000,000+ - 1,000,000,000+: 1367 Responses (100 perc)

As you can see this does not work out well, but we can now have a starting point at least.  If we start the label of 
success at anything 10,000,000+ or above, we now know we are grabbing the top 43.6% of apps as far as installs go.

In [ ]:
succ_group = [k for k, v in installs.items() if v[0]>=13]  # Create binary label for any install category with index 16+

df['label'] = df.Installs.isin(succ_group)  # Attach labels to the dataframe

# df.to_csv(r'RawData/google_labeled.csv', index=False)
df.head()

## Analyzing our Data
Now, lets look at the labels for success that we defined and see if we can gain any insight into what type of apps meet 
our criteria.

In [ ]:
temp = df[['Category','label']].groupby('Category').agg(['mean', 'count'])  # Group and Aggregate on our label
temp.sort_values(('label', 'mean'), ascending=False)[:5]  # Sort and show results

As we can see from a basic average of our labels the top 5 app Categories are:
- Photography
- Entertainment
- Game
- Education
- Weather

This is showing that our top categories have a "success" rate of approx. 67 - 87%
It is also to note the sizes of each group.  While they are not our largest categories I believe they 
are of sufficient size for this analysis.

Now we will move onto doing this same process for some of our other features

In [ ]:
temp = df[['Content Rating','label']].groupby('Content Rating').agg(['mean', 'count'])  # Group and aggregate on label
temp.sort_values(('label', 'mean'), ascending=False)[:5]  # Sort and show results

Of interest, it seems that games rated for young kids (teens) appear to have the highest chance of success.
Though due to the unbalanced sample sizes it may only be fair to say that having a rating outside of 
'Everyone' is the best practice.  Due to this I would suggest some level of mature content appropriate for 
young teens.

In [ ]:
temp = df[['Size_Class','label']].groupby('Size_Class').agg(['mean', 'count'])  # Group and aggregate on our label
temp.sort_values(('label', 'mean'), ascending=False)  # Sort & show results

We can see clearly there are two things going on here. The smallest Apps (Category 1) appear to be very 
successful at being installed.  It seems likely that these are utility apps, meant to accomplish one small 
goal.  After that there is a clear pattern that the larger Apps appear to be the most successful.

So my take away here is that there are probably two types of successful app strategies, create a utility app 
that provides small amounts of functionality OR build an large app that provides a very large experience.


In [ ]:
temp = df[['rating_round','label']].groupby('rating_round').agg(['mean', 'count'])  # Group & Agg on our label
temp.sort_values(('label', 'mean'), ascending=False)  # Sort and show results

Interestingly we can see that ratings seems to have a strong correlation with our success criteria except
the 5.0 ratings appear to be throwing it off.  It seems fair to say that the only apps that receive such high
ratings are those that do not have a lot of opinions (low reviews and installs).

Moving on to dig into the genre category is more complicated as some apps can have multiple genres.  We
already handled this using one hot encoding, but not lets look at the success rates of apps depending on
genres.  We will be removing any genre that doesn't have at least 100 apps to limit the categories.

In [ ]:
genres = df.loc[:, df.columns.str.startswith('Genre_')].columns  # Genre Column list
results = {}  # Init dict
for i, col in enumerate(genres):  # Loop through columns
    temp = df[df[col] == 1]  # init temp df containing only that genre's apps
    met = temp.label.mean()  # Establish the mean of the success labels
    cnt = len(temp)  # establish how many apps fit that genre
    if cnt < 100:  # Filter our results to only contain 100 or more apps
        continue
    else:
        results[col] = [met, cnt]  # create a key and append values

# Sort the dict by it's first item in reverse order
results = {k: v for k, v in sorted(results.items(), key=lambda item: item[1],reverse=True)}
results  # Preview results

We can now see that certain genres appear to be very successful in garnering installs at a high rate:
- Racing
- Arcade
- Action
- Photography
- Casual

## Initial Findings
Through our cleaning, feature engineering and basic analysis we can clearly say that we have the ability to
narrow down the scope to what types of apps would be most successful.  For example, we know that a **Casual Racing
Game App that is large (offer a very robust experience) and has good early reviews should be quite successful**

However, our goal isn't to just make one suggestion to our designers but to be able to make numerous or even
take ideas and provide feedback on whether we believe it will garner a lot of installs or not.  To do this
we will take our labels of success and our features and see if we can establish a predictive model that could
take basic features of the App including reviews from a test group and determine if the app is likely to be
installed at a large scale.

## Predictive Model

Now it is time to trim off the unnecessary columns from our data set so that we can convert our df into valid arrays 
for our model.  We will be removing both size_cat and rating_round as they are likely to be
collinear with size_class & ratings,  This will lower the sparsity of the data and should result
in a more efficient model.

It is also important to determine what we would consider to be successful for our model.  In this situation
it is always good to know what type of model we would like:
- Sensitive
- Specific

This will let us know how to tune our model.  In this case it seems to be more common to be looking
for a specific model.  This means that we will not often make False predictions of success but
this may come at the cost of missing some successful apps.  This ensures that we are very confident
of our prediction for success, but not necessarily confident in our prediction of unsuccessful.

In this case it makes sense that we want to be correct when we predict success at least 85% of the time,
but we also want to make sure we are not missing too many successful app ideas, meaning we will
still want a decent sensitivity (maybe 50%).


In [ ]:
remove = ['App', 'Category', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated',
          'Current Ver', 'Android Ver', 'Reviews', 'rating_round', 'Size_Class']  # Establish list of unnecessary cols


df.drop(remove, axis=1, inplace=True)  # Drop Col list
df.head()

In this case I usually find it valuable to start with a simple model such as a Decision tree to
evaluate the likelihood of success.  From there we can go more advance such as a random forest, but
I want to ensure to start as a decision tree also for its ability to return feature importance.

Even if we cannot build a successful model, the feature importance can prove as useful insight for
our app developers even if this is too complicated of a problem for our current dataset.

### Decision Tree
We will first separate our labels from our DF so that we can split the data by the 80/20 rule.
This will provide us with both datasets for training the model and testing it's performance, but
also the labels to do so.


In [ ]:
RSEED=500  # Establish a seed

# Create our Labels
labels = np.array(df.pop('label'))

# Split our data by 80/20 rule
train, test, train_labels, test_labels = train_test_split(df, labels, stratify=labels, test_size=0.20, random_state=RSEED)

In our care the only issue we may have with our data is missing values in the rating field.  There
are many ways to handle this but for this project I think it makes most sense to replace those missing ratings
with the average results for each set.

In [ ]:
# Impute ratings column for missing values
train.Rating = train.Rating.fillna(train.Rating.mean())
test.Rating = test.Rating.fillna(test.Rating.mean())

# Features for feature importances
features = list(train.columns)

In [ ]:
# Preview the size of our train set
train.shape

In [ ]:
# Preview the size of our test set
test.shape

Now that we have prepared our data, it is time to train the tree


In [ ]:
# train the tree
tree = DecisionTreeClassifier(random_state=RSEED)  # Establish a tree using our seed

tree.fit(train, train_labels)  # Train the tree

# Basic Tree Information
print(f'Decision tree has {tree.tree_.node_count} nodes with maximum depth {tree.tree_.max_depth}.')

We will now make predictions and grab the predicted probabilities for both sets


In [ ]:
# Establish the probabilities of fitting into the class
train_probs = tree.predict_proba(train)[:, 1]
probs = tree.predict_proba(test)[:, 1]

# Make the predictions
train_predictions = tree.predict(train)
predictions = tree.predict(test)

Now we can review our initial accuracy scores and evaluate our model's performance.  We will create
a function to evaluate our models performance because we will have to refer back to it several times
throughout this project.


In [ ]:
"""Compare machine learning model to baseline performance.
Computes statistics and shows ROC curve."""
def evaluate_model(predictions, probs, train_predictions, train_probs):
    """
    :param predictions: Test Predicitions from our model
    :param probs: Test probabilities from our model
    :param train_predictions: Train Predicitions from our model
    :param train_probs: Train probabilities from our model
    :return: None

    This will take our test results from our model and compare them to baseline and training
    results.  it will calculate precision and recall (specificity and Sensitivity) as well
    as display the ROC curve
    """
    baseline = {}
    
    baseline['recall'] = recall_score(test_labels, [1 for _ in range(len(test_labels))])
    baseline['precision'] = precision_score(test_labels, [1 for _ in range(len(test_labels))])
    baseline['roc'] = 0.5
    
    results = {}
    
    results['recall'] = recall_score(test_labels, predictions)
    results['precision'] = precision_score(test_labels, predictions)
    results['roc'] = roc_auc_score(test_labels, probs)
    
    train_results = {}
    train_results['recall'] = recall_score(train_labels, train_predictions)
    train_results['precision'] = precision_score(train_labels, train_predictions)
    train_results['roc'] = roc_auc_score(train_labels, train_probs)
    
    for metric in ['recall', 'precision', 'roc']:
        print(f'{metric.capitalize()} Baseline: {round(baseline[metric], 2)} Test: {round(results[metric], 2)} Train: {round(train_results[metric], 2)}')
    
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(test_labels, [1 for _ in range(len(test_labels))])
    model_fpr, model_tpr, _ = roc_curve(test_labels, probs)
    
    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 16
    
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend()
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curves');



In [ ]:
evaluate_model(predictions, probs, train_predictions, train_probs)

Now that we have a baseline of accuracy for our model it is often useful to visualize the confusion
matrix for review of our findings.  We will create a function to do this as we will refer back to
this several times throughout the project.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Oranges):
    """
    :param cm: Our Confusion Matric
    :param classes: Our Classes we are prediciting
    :param normalize: Represent as a range from 0 - 1
    :param title: Title of the plot
    :param cmap: Color Map to plot with
    :return: Returns the TP, FP, TN, FN results from our Confuision Matrix
     This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    if normalize:  # Establish whether to normalize or not
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # print(cm)

    # plt confusion matrix
    plt.figure(figsize = (10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14)

    # Establishes format
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    # Labeling the plot
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize = 20,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size = 18)
    plt.xlabel('Predicted label', size = 18)
    
    return cm[1, 1], cm[0,0], cm[0, 1], cm[1, 0]

In [ ]:
# Init a Confusion Matrix from our test results
cm = confusion_matrix(test_labels, predictions)

# Display the confusion Matrix and return the Quadrant results
TP, TN, FP, FN = plot_confusion_matrix(cm, classes = ['Unsuccessful App', 'Successful App'],
                      title = 'App Confusion Matrix')

# Print Classification Error Score
print('Overall, how often the model is incorrect.',os.linesep, 'Classification Error: ', ((FP + FN)/float(TP + TN + FP + FN)))

Now that we understand our model's performance we can start looking into which features drove
our models decisions the most.  To do this we need to compute the feature importance for each
feature.  Thankfully decision trees have this feature built into their library

In [ ]:
# Create a feature importance dataframe for review
fi = pd.DataFrame({'feature': features,
                   'importance': tree.feature_importances_}).\
                    sort_values('importance', ascending = False)
fi.head()


#TODO: Now we need to explain our initial results and describe how they may be promising
then we need to explain our ensemble model to improve performance, etc..
















In [ ]:
# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               random_state=RSEED, 
                               max_features = 'sqrt',
                               n_jobs=-1, verbose = 1)

# Fit on training data
model.fit(train, train_labels)

In [ ]:
n_nodes = []
max_depths = []

for ind_tree in model.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = model.predict(train)
train_rf_probs = model.predict_proba(train)[:, 1]

rf_predictions = model.predict(test)
rf_probs = model.predict_proba(test)[:, 1]

In [ ]:
evaluate_model(rf_predictions, rf_probs, train_rf_predictions, train_rf_probs)

In [ ]:

cm = confusion_matrix(test_labels, rf_predictions)
TP, TN, FP, FN = plot_confusion_matrix(cm, classes = ['Unsuccessful App', 'Successful App'],
                      title = 'App Confusion Matrix')
print('Overall, how often the model is incorrect.',os.linesep, 'Classification Error: ', ((FP + FN)/float(TP + TN + FP + FN)))


In [ ]:
fi_model = pd.DataFrame({'feature': features,
                   'importance': model.feature_importances_}).\
                    sort_values('importance', ascending = False)
fi_model.head(10)


In [ ]:
# Hyperparameter grid
param_grid = {
    'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
estimator = RandomForestClassifier(random_state = RSEED)

# Create the random search model
rs = RandomizedSearchCV(estimator, param_grid, n_jobs = -1, 
                        scoring = 'roc_auc', cv = 3, 
                        n_iter = 10, verbose = 1, random_state=RSEED)

# Fit 
rs.fit(train, train_labels)

In [ ]:
rs.best_params_


In [ ]:
best_model = rs.best_estimator_

In [ ]:
train_rf_predictions = best_model.predict(train)
train_rf_probs = best_model.predict_proba(train)[:, 1]

rf_predictions = best_model.predict(test)
rf_probs = best_model.predict_proba(test)[:, 1]

In [ ]:
n_nodes = []
max_depths = []

for ind_tree in best_model.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')


In [ ]:
evaluate_model(rf_predictions, rf_probs, train_rf_predictions, train_rf_probs)

In [ ]:
cm = confusion_matrix(test_labels, rf_predictions)
TP, TN, FP, FN = plot_confusion_matrix(cm, classes = ['Unsuccessful App', 'Successful App'],
                      title = 'App Confusion Matrix')
print('Overall, how often the model is incorrect.',os.linesep, 'Classification Error: ', ((FP + FN)/float(TP + TN + FP + FN)))


In [ ]:
fpr, tpr, thresholds = roc_curve(test_labels, rf_probs)

def evaluate_thresh(threshold):
    print('Recall: ', round(tpr[thresholds > threshold][-1], 3))
    print('Precision: ', round(1 - fpr[thresholds > threshold][-1], 3))
    print(os.linesep)

for i in range(20, 80, 5):
    print('Threshold: ' + str(round(i * 0.01, 2)))
    evaluate_thresh(i*0.01)

In [ ]:
# Goal of 85% Precision
new_threshold = 0.6

y_pred_class = binarize(rf_probs.reshape(-1, 1), threshold=new_threshold)
cm = confusion_matrix(test_labels, y_pred_class)
TP, TN, FP, FN = plot_confusion_matrix(cm, classes = ['Unsuccessful App', 'Successful App'],
                      title = 'App Confusion Matrix')
print('Overall, how often the model is incorrect.', os.linesep, 'Classification Error: ', ((FP + FN)/float(TP + TN + FP + FN)))
print('Recall: ', TP / (FN + TP))
print('Precision: ', TN / (TN + FP))

